In [103]:
from IPython.display import JSON
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
import os.path as path
import random
import glob
import time
from selenium import webdriver

## Fetch Data from NOAA?
This isn't working for me - as soon as I put in dates I get no data in the response :-(

In [ ]:
from noaa_sdk import noaa
n = noaa.NOAA()
# spec: https://www.weather.gov/documentation/services-web-api#/

In [ ]:
observations = n.stations_observations('KBOS', start="2021-04-01", end="2021-04-02")

In [ ]:
observations = n.get_observations('11365','US',start='2020-04-01',end='2020-04-02')

In [ ]:
observations = [o for o in observations]
print(len(observations))

In [ ]:
def format_url(station_id, start_date, end_date):
    params = dict(start="{}T00:00:00Z".format(start_date[:10]), end="{}T00:00:00Z".format(end_date[:10]))
    url = "https://api.weather.gov/stations/{}/observations".format(station_id)
    return requests.get(url, params=params)

In [ ]:
r = format_url('KBOS', "2021-04-05", "2021-04-06")

In [ ]:
r.json()

## Scrape data from Weather Underground?

In [110]:
station = 'KBOS'

In [116]:
dates_to_fetch = []
for year in [2020, 2021]:
    start_date = dt.date(year,5,4)
    for d in range(1,65):
        day = start_date - dt.timedelta(days=d)
        dates_to_fetch.append(day)
print(len(dates_to_fetch))

128


In [130]:
def grab_data_for_dates(dates):
    # the tables are loaded via async requests to the server, so we have to use selenium here not requests
    browser=webdriver.Firefox()

    for day in dates:
        date_str = "{}-{}-{}".format(day.year,day.month,day.day)
        url = "https://www.wunderground.com/history/daily/{}/date/{}".format(station,date_str)
        cache_file = path.join("data", "{}-{}.html".format(station,date_str))
        if path.exists(cache_file):
            #print("{} - cached already".format(date_str))
            continue
        print("{} - fetching".format(date_str))
        browser.get(url)
        html = browser.page_source
        with open(cache_file, "w") as f:
            f.write(html)
        time.sleep(random.randrange(2,5))

    browser.close()

In [129]:
grab_data_for_dates(dates_to_fetch)

2020-4-18 - fetching


## Parse the data out of the cached files

In [127]:
datafile = "{}-historical-temps.csv".format(station)

In [158]:
data = []
for f in glob.glob(path.join("data", "*.html")):
    name_parts = f.replace(".html","").split("-")
    date = dt.date(int(name_parts[1]),int(name_parts[2]),int(name_parts[3]))
    with open(f, 'r') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        day_observations = []
        hours_done = []
        daily_observations_table = soup.find_all('table')[1]
        rows = daily_observations_table.find_all('tr')
        for row in rows[1:]:
            cells = row.find_all("td")
            if len(cells) == 10:
                time=cells[0].get_text()
                hour=int(time.split(':')[0])
                if 'AM' in time:
                    hour = 0 if hour == 12 else hour
                else:
                    hour = hour + 12
                data_row = dict(
                    date=str(date),
                    time=time,
                    hour=hour,
                    temperature=cells[1].get_text().replace("\xa0F",""),
                    dew_point=cells[2].get_text().replace("\xa0F",""),
                    humidity=cells[3].get_text().replace("\xa0%",""),
                    wind=cells[4].get_text(),
                    wind_speed=cells[5].get_text().replace("\xa0mph",""),
                    wind_gust=cells[6].get_text().replace("\xa0mph",""),
                    pressure=cells[7].get_text().replace("\xa0in",""),
                    precipitation=cells[8].get_text().replace("\xa0in",""),
                    condition=cells[9].get_text(),
                )
                # now we need to limit to one observation per hour - just keep the first one
                if hour not in hours_done:
                    hours_done.append(hour)
                    day_observations.append(data_row)
        data += day_observations
    except Exception as e:
        print(f)
        print(e)
df = pd.DataFrame(data)
df.to_csv(datafile)

In [161]:
df = pd.read_csv(datafile)
df.head()

,date,time,hour,temperature,dew_point,humidity,wind,wind_speed,wind_gust,pressure,precipitation,condition
0,4/11/20,12:54 AM,0,40,29,65,WNW,15,23,29.56,0.0,Fair
1,4/11/20,1:54 AM,1,39,29,67,W,13,22,29.57,0.0,Fair
2,4/11/20,2:54 AM,2,39,29,67,W,16,0,29.59,0.0,Fair
3,4/11/20,3:54 AM,3,39,26,60,W,15,23,29.61,0.0,Fair
4,4/11/20,4:54 AM,4,38,26,62,W,10,0,29.63,0.0,Fair
